In [17]:
# import sys, datetime
import serial.tools.list_ports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import time, os
import crcmod
from datetime import datetime

In [18]:
# Выбор порта данных с катушек
ports = serial.tools.list_ports.comports()
port = None
# print(ports)
for i, port in enumerate(ports):
    # print(f"{port}")
    print(f"Порт {i}: {port.device}")
    # print(f"Описание: {port.description}")
    # print(f"VID:PID: {port.vid}:{port.pid}")
if port:
    # prt = int(input('Какой порт?'))
    port = ports[1].device
    port_motor = ports[0].device
    print(f'Для получения данных выбран порт - {port}')
    print(f'Для двигателя выбран порт - {port_motor}')

Порт 0: COM1
Порт 1: COM5
Для получения данных выбран порт - COM5
Для двигателя выбран порт - COM1


Настройки COM-порта: такие же как и были, но скорость 3000000 бод/с.

Запрос на запуск считывания: R300000;4194304\n (первое число - количество выборок/c, целое от 1 до 300000 (если указать больше, то всё равно будет 300к); второе число - количество выборок от 1 до 4194304). В ответ сразу приходит строка "F=300 kSPS,N=4194304\n" в которой указано какая в точности частота будет использоваться для тактирования и сколько выборок будет сделано. При удачном завершении считывания придёт строка "1\n".

Запрос на запуск передачи: S0;4194304\n (первое число - номер выборки с которой начать передачу; второе число - количество выборок для передачи). В ответ приходят данные от каждой платы по очереди. Данные содержат заголовок, например "ADC001" или "ENC001", затем массив данных, в конце контрольная сумма на массив данных CRC16_CCIT_ZERO (полином 0x1021, начальное значение - 0).

In [86]:
BAUDRATE = 2000000
# BAUDSPEED = 300000
BAUDSPEED = 100000
# N_SAMPLES = 4194304
N_SAMPLES = 250000

In [ ]:
# Чтение данных с катушек в память контроллера
with (serial.Serial(port, baudrate=BAUDRATE, bytesize=8, stopbits=1, timeout=None)) as serialData:
    # Read data from Sensor
    command = f'R{BAUDSPEED};{N_SAMPLES}\n'
    # Send the command to the DataPort
    # serialData.write(command.encode(encoding="utf-8"))
    serialData.write(command.encode())
    dataRead = serialData.readline()
    dataReady = serialData.readline()

print(f'{dataRead}')
print(f'{dataReady}')  


b'F=100 kSPS,N=250000\n'
b'1\n'


In [93]:
TIMEBASE_CONSTANT = int(float(dataRead.decode().split('F=')[1].split()[0].replace(',','.'))*1000)  # Постоянная времени системы [мкс]
TIMEBASE_CONSTANT

100000

In [ ]:
N_SAMPLES_STEP = 1000
ADC = b''
EDC = b''

# Создаём объект CRC-16-CCITT-ZERO
crc16_func = crcmod.mkCrcFun(
    poly=0x11021,      # Полином: x^16 + x^12 + x^5 + 1 (0x1021, но с битом переноса)
    initCrc=0x0000,  # Начальное значение — 0x0000 (ZERO)
    xorOut=0x0000,   # Окончательный XOR — 0x0000
    rev=False          # Прямой порядок битов (normal)
)

# Чтение данных из памяти контроллера
with (serial.Serial(port, baudrate=BAUDRATE, bytesize=8, stopbits=1, timeout=1)) as serialData:
    for i in range(0, int(N_SAMPLES), N_SAMPLES_STEP):

        command = f'S{i};{N_SAMPLES_STEP}\n'
        serialData.write(command.encode())

        lineADC = serialData.read(2*N_SAMPLES_STEP+9)
        dataADC = lineADC[6:-3]
        crcADC = lineADC[-3:-1]
        if crc16_func(dataADC) != int.from_bytes(crcADC, 'big'):
            print('DataADC not ok')
        else:
            ADC += dataADC

        lineEDC = serialData.read(2*N_SAMPLES_STEP+9)
        dataEDC = lineEDC[6:-3]
        crcEDC = lineEDC[-3:-1]
        if crc16_func(dataEDC) != int.from_bytes(crcEDC, 'big'):
            print('DataEDC not ok')
        else:
            EDC += dataEDC

In [71]:
rawADC = np.frombuffer(ADC, dtype=np.uint8)
data = rawADC.view(dtype='>i2').astype(np.int16)
rawEDC = np.frombuffer(EDC, dtype=np.uint8)
encoder = rawEDC.view(dtype='>i2').astype(np.int16)

len(data) == len(encoder)

250000